# **Marketing Intelligence: Data Pipeline for Consumer Behavior in PostgreSQL**

**by: Yunido Baheramsyah**

This project aims to build a data pipeline for analyzing consumer behavior using PostgreSQL and Pandas. The pipeline will include data cleaning, normalization (up to 3NF), exporting to CSV, and creating SQL schemas with query testing.


---

# **I. INTRODUCTION**

#### **I.A. Background**

Modern marketing teams rely on timely, trustworthy consumer-behavior data to allocate budgets, optimize campaigns, and personalize offers. In practice, source files often arrive as ad-hoc CSV exports with inconsistent headers, mixed data types, and missing values, making analysis slow, error-prone, and hard to reproduce. Without a governed schema and shared definitions, core KPIs (e.g., total purchase amount, average order value) are computed differently across teams, leading to misalignment and suboptimal decisions.

This project transforms a raw consumer-behavior CSV into an analytics-ready, governed dataset. Using Pandas, we standardize column names, clean and cast data types, handle nulls, and derive features (e.g., purchase-amount bands) for downstream analysis. We then model the data in PostgreSQL using a 3NF design with conformed dimensions (time, gender, payment method, product category, location) and a purchase fact table—ensuring referential integrity, consistent joins, and minimal redundancy. The result is a reliable foundation for marketing use cases such as segmentation, cohort and funnel analysis, A/B test readouts, campaign attribution, and KPI tracking, while enabling repeatable loads via SQL and clear hand-offs to BI tools.



#### **I.B. Setup & Objective**

This notebook is designed to be run end-to-end:
- Data exploration (brief EDA).

- Data cleaning and type standardization.

- Creation of main_table (10 columns) as instructed.

- Normalization to 3NF into 6 tables (1 fact + 5 dimensions).

- Export all tables to CSV for the COPY process into PostgreSQL.

- Automatically generate DDL + COPY into a .sql file.

- Run 2 SQL tests (written in markdown) and export their results to two separate CSV files.

**Important note**: Column names in public datasets are sometimes inconsistent. To be robust,
this notebook performs column-name normalization and heuristic detection of target columns
(e.g., purchase_amount might appear as purchase amount, amount, price, etc.).

---

# **II. IMPORT LIBRARIES & DATA LOADING**

In [1]:

import pandas as pd             # Tabular data analysis/manipulation (DataFrame ops, read_csv, joins, groupby, etc.)
import numpy as np              # Numerical computing & vectorized operations (arrays, math/stat funcs, np.where)
from pathlib import Path        # Object-oriented file paths (create folders, read/write files in a cross-platform way)
import re                       # Regular expressions for pattern matching/cleaning text (e.g., strip currency symbols)
import json                     # Serialize/deserialize Python objects to/from JSON (configs, metadata, results)
import textwrap                 # Format multi-line strings; dedent/indent blocks (useful for generating SQL/Markdown)
from datetime import datetime   # Parse/format timestamps and handle date-time comparisons/conversions

# Load Data
PROJECT_DIR = Path.cwd() / "gc2_consumer_behavior_project"
TABLES_DIR = PROJECT_DIR / "tables"
QUERY_OUT_DIR = PROJECT_DIR / "query_outputs"
for d in [PROJECT_DIR, TABLES_DIR, QUERY_OUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DATA_URL = "https://raw.githubusercontent.com/FTDS-learning-materials/phase-0/refs/heads/main/src/Consumer_Behavior_Analysis_Data(GC2Set2).csv"


---

# **III. DATA EXPLORATION**


#### **III.A. Load & display a few rows of data**

In [2]:

def _to_snake(s: str) -> str:
    s = s.strip()
    s = re.sub(r"[^\w]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s.lower()

df_raw = pd.read_csv(DATA_URL)
df = df_raw.copy()
df.columns = [_to_snake(c) for c in df.columns]

print("Shape:", df.shape)
display(df.head(10))


Shape: (1000, 26)


,age,gender,income_level,marital_status,education_level,occupation,purchase_category,purchase_amount,frequency_of_purchase,purchase_channel,...,customer_satisfaction,engagement_with_ads,device_used_for_shopping,payment_method,time_of_purchase,discount_used,customer_loyalty_program_member,purchase_intent,shipping_preference,time_to_decision
0,22.0,Female,Middle,Married,Bachelor's,Middle,Gardening & Outdoors,$333.80,4.0,Mixed,...,7.0,NaN,NaN,Credit Card,3/1/2024,True,False,Need-based,No Preference,NaN
1,NaN,NaN,High,Married,High School,NaN,Food & Beverages,$222.22,11.0,In-Store,...,5.0,High,Tablet,PayPal,4/16/2024,True,NaN,Wants-based,Standard,6.0
2,24.0,Female,Middle,Single,Master's,High,Office Supplies,$426.22,2.0,Mixed,...,7.0,Low,Smartphone,Debit Card,3/15/2024,True,True,Impulsive,No Preference,3.0
3,29.0,Female,Middle,Single,Master's,Middle,Home Appliances,$101.31,6.0,Mixed,...,1.0,NaN,Smartphone,Other,10/4/2024,True,True,Need-based,Express,10.0
4,33.0,Female,Middle,Widowed,High School,Middle,Furniture,$211.70,6.0,Mixed,...,10.0,NaN,NaN,Debit Card,1/30/2024,False,NaN,Wants-based,No Preference,4.0
5,45.0,Male,Middle,Married,Master's,High,Office Supplies,$487.95,8.0,Mixed,...,3.0,NaN,Tablet,Debit Card,3/19/2024,False,False,Planned,No Preference,7.0
6,21.0,Female,Middle,Divorced,High School,Middle,Furniture,$486.40,12.0,NaN,...,9.0,Low,Desktop,Debit Card,3/15/2024,True,False,Need-based,Express,13.0
7,39.0,Male,Middle,Single,High School,Middle,Books,$218.06,6.0,Online,...,9.0,NaN,Desktop,Credit Card,3/17/2024,False,True,Impulsive,No Preference,NaN
8,24.0,Female,High,Divorced,Master's,NaN,Office Supplies,$201.96,NaN,NaN,...,2.0,High,Desktop,Cash,6/10/2024,False,False,Need-based,Standard,7.0
9,25.0,Bigender,NaN,Married,NaN,High,Sports & Outdoors,$418.83,7.0,Online,...,5.0,Medium,Tablet,PayPal,10/7/2024,True,False,Need-based,No Preference,13.0


#### **III.B. Summary & Initial Findings**

In [3]:

display(df.info())
display(df.describe(include='all').T)
print("\nMissing values per column:")
display(df.isna().sum().sort_values(ascending=False))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   943 non-null    float64
 1   gender                                954 non-null    object 
 2   income_level                          939 non-null    object 
 3   marital_status                        955 non-null    object 
 4   education_level                       937 non-null    object 
 5   occupation                            952 non-null    object 
 6   purchase_category                     958 non-null    object 
 7   purchase_amount                       946 non-null    object 
 8   frequency_of_purchase                 960 non-null    float64
 9   purchase_channel                      958 non-null    object 
 10  brand_loyalty                         958 non-null    float64
 11  product_rating    

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,943.0,NaN,NaN,NaN,34.288441,9.35928,18.0,26.0,34.0,42.0,50.0
gender,954,8,Female,433,NaN,NaN,NaN,NaN,NaN,NaN,NaN
income_level,939,2,High,485,NaN,NaN,NaN,NaN,NaN,NaN,NaN
marital_status,955,4,Widowed,253,NaN,NaN,NaN,NaN,NaN,NaN,NaN
education_level,937,3,Bachelor's,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN
occupation,952,2,High,491,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purchase_category,958,24,Electronics,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purchase_amount,946,938,$490.41,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
frequency_of_purchase,960.0,NaN,NaN,NaN,6.932292,3.145515,2.0,4.0,7.0,10.0,12.0
purchase_channel,958,3,Mixed,329,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Missing values per column:


engagement_with_ads                     288
social_media_influence                  281
education_level                          63
income_level                             61
discount_sensitivity                     59
time_to_decision                         59
customer_satisfaction                    57
age                                      57
product_rating                           54
purchase_amount                          54
time_spent_on_product_research_hours     53
customer_loyalty_program_member          53
shipping_preference                      51
device_used_for_shopping                 50
occupation                               48
purchase_intent                          47
gender                                   46
marital_status                           45
discount_used                            45
payment_method                           43
purchase_category                        42
purchase_channel                         42
brand_loyalty                   

#### **III.C. Column List & Next Steps**

In [4]:

columns = list(df.columns)
print(columns)


['age', 'gender', 'income_level', 'marital_status', 'education_level', 'occupation', 'purchase_category', 'purchase_amount', 'frequency_of_purchase', 'purchase_channel', 'brand_loyalty', 'product_rating', 'time_spent_on_product_research_hours', 'social_media_influence', 'discount_sensitivity', 'return_rate', 'customer_satisfaction', 'engagement_with_ads', 'device_used_for_shopping', 'payment_method', 'time_of_purchase', 'discount_used', 'customer_loyalty_program_member', 'purchase_intent', 'shipping_preference', 'time_to_decision']


#### **III.D. Unique Values for Categorical Columns**

In [5]:

# Categorical Columns and Numerical Columns
cat_cols = [c for c in df.columns if df[c].dtype == 'object' or df[c].nunique() <= 30]
num_cols = [c for c in df.columns if c not in cat_cols]

for c in cat_cols[:12]:
    print(f"\n-- {c} --")
    print(sorted(df[c].dropna().astype(str).unique()[:20]))
    
cat_cols, num_cols[:10]



-- gender --
['Agender', 'Bigender', 'Female', 'Genderfluid', 'Genderqueer', 'Male', 'Non-binary', 'Polygender']

-- income_level --
['High', 'Middle']

-- marital_status --
['Divorced', 'Married', 'Single', 'Widowed']

-- education_level --
["Bachelor's", 'High School', "Master's"]

-- occupation --
['High', 'Middle']

-- purchase_category --
['Animal Feed', 'Arts & Crafts', 'Baby Products', 'Books', 'Clothing', 'Electronics', 'Food & Beverages', 'Furniture', 'Gardening & Outdoors', 'Groceries', 'Health Care', 'Health Supplements', 'Home Appliances', 'Luxury Goods', 'Mobile Accessories', 'Office Supplies', 'Packages)', 'Software & Apps', 'Sports & Outdoors', 'Toys & Games']

-- purchase_amount --
['$101.31 ', '$109.29 ', '$201.96 ', '$211.70 ', '$218.06 ', '$222.22 ', '$236.99 ', '$238.93 ', '$333.80 ', '$356.45 ', '$389.70 ', '$418.83 ', '$426.22 ', '$454.39 ', '$463.07 ', '$486.40 ', '$487.95 ', '$490.75 ', '$58.51 ', '$79.81 ']

-- frequency_of_purchase --
['10.0', '11.0', '12.0',

(['gender',
  'income_level',
  'marital_status',
  'education_level',
  'occupation',
  'purchase_category',
  'purchase_amount',
  'frequency_of_purchase',
  'purchase_channel',
  'brand_loyalty',
  'product_rating',
  'time_spent_on_product_research_hours',
  'social_media_influence',
  'discount_sensitivity',
  'return_rate',
  'customer_satisfaction',
  'engagement_with_ads',
  'device_used_for_shopping',
  'payment_method',
  'time_of_purchase',
  'discount_used',
  'customer_loyalty_program_member',
  'purchase_intent',
  'shipping_preference',
  'time_to_decision'],
 ['age'])

---

# **IV. DATA CLEANING**

**Brief Instructions:**
- Column slicing: `time_of_purchase`, `gender`, `purchase_amount`, `age` + 3 categorical columns + 3 numeric columns > total 10 columns > `main_table`.
- Data type fixes (remove currency symbols; ensure proper numeric/integer casting).
- Missing-value handling (for integer fields > impute 0).
- Create a new categorical column: `categorical_purchase_amount` (low < 150, mid 150–<300, high ≥ 300). Boundary note: values =150 go to mid, and =300 go to high.
- Standardize `gender` to Male/Female and drop rows outside these labels.
- Clean whitespace in column names & convert `time_of_purchase` to datetime.


In [6]:
# Column slicing
def find_col(candidates):
    for cand in candidates:
        for col in df.columns:
            if cand in col:
                return col
    return None

# Find and validate required columns
col_time  = find_col(["time_of_purchase","purchase_time","time","timestamp","date"])
col_gender= find_col(["gender","sex"])
col_amount= find_col(["purchase_amount","amount","price","spent","spend"])
col_age   = find_col(["age"])

# Validate all required columns are found
if not all([col_time, col_gender, col_amount, col_age]):
    raise RuntimeError(f"Tidak semua kolom wajib ditemukan: time={col_time}, gender={col_gender}, amount={col_amount}, age={col_age}")
# Identify additional categorical and numeric columns
cat_pool = [c for c in df.columns if c not in [col_time,col_gender,col_amount,col_age] and (df[c].dtype=='object' or df[c].nunique()<=30)]
num_pool = [c for c in df.columns if c not in [col_time,col_gender,col_amount,col_age] and c not in cat_pool]

# Identify additional categorical and numeric columns
extra_cats = cat_pool[:3]
extra_nums = num_pool[:3]

# Combine all selected columns
selected_cols = [col_time,col_gender,col_amount,col_age] + extra_cats + extra_nums
selected_cols = selected_cols[:10]
main_table = df[selected_cols].copy()

# Helper function to convert columns to numeric safely
def to_numeric_safely(s):
    cleaned = s.astype(str).str.replace(r"[^0-9\.-]", "", regex=True)
    # Use pd.to_numeric for robust conversion, errors='coerce' will set invalid parsing to NaN
    return pd.to_numeric(cleaned, errors="coerce")

main_table[col_amount] = to_numeric_safely(main_table[col_amount])
main_table[col_age] = pd.to_numeric(main_table[col_age], errors="coerce").fillna(0).astype(int)

# Fill NaN values in numeric columns with 0
def bucket_amount(x):
    if pd.isna(x):
        return pd.NA
    if x < 150:
        return "low"
    elif x < 300:
        return "mid"
    else:
        return "high"

main_table["categorical_purchase_amount"] = main_table[col_amount].apply(bucket_amount)

# Normalize gender values
def norm_gender(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().lower()
    if s in ["male","m","laki-laki","pria"]: return "Male"
    if s in ["female","f","perempuan","wanita"]: return "Female"
    return "OTHER"

main_table[col_gender] = main_table[col_gender].apply(norm_gender)
before = len(main_table)
main_table = main_table[main_table[col_gender].isin(["Male","Female"])].copy()
after = len(main_table)
print(f"Dropped non-binary/unknown genders: {before-after} rows")

main_table[col_time] = pd.to_datetime(main_table[col_time], errors="coerce", dayfirst=False, infer_datetime_format=True)

display(main_table.head())
display(main_table.info())


Dropped non-binary/unknown genders: 141 rows


C:\Users\User\AppData\Local\Temp\ipykernel_13380\1346115019.py:67: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  main_table[col_time] = pd.to_datetime(main_table[col_time], errors="coerce", dayfirst=False, infer_datetime_format=True)


,time_of_purchase,gender,purchase_amount,age,income_level,marital_status,education_level,categorical_purchase_amount
0,2024-03-01,Female,333.80,22,Middle,Married,Bachelor's,high
2,2024-03-15,Female,426.22,24,Middle,Single,Master's,high
3,2024-10-04,Female,101.31,29,Middle,Single,Master's,low
4,2024-01-30,Female,211.70,33,Middle,Widowed,High School,mid
5,2024-03-19,Male,487.95,45,Middle,Married,Master's,high


<class 'pandas.core.frame.DataFrame'>
Index: 859 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   time_of_purchase             831 non-null    datetime64[ns]
 1   gender                       859 non-null    object        
 2   purchase_amount              812 non-null    float64       
 3   age                          859 non-null    int32         
 4   income_level                 807 non-null    object        
 5   marital_status               821 non-null    object        
 6   education_level              803 non-null    object        
 7   categorical_purchase_amount  812 non-null    object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 57.0+ KB


None


#### **IV.A. Data Normalization (up to 3NF)**
Target: 6 tables --> 1 fact (`main_fact`) + 5 dimensions.
- `dim_gender`
- `dim_payment_method` (if available; fallback to another categorical column)
- `dim_product_category` (if available; fallback to another categorical column)
- `dim_location` (combine city/state/country if present; otherwise use a single categorical column)
- `dim_time` (derived from the time column)
- `main_fact` (default PK: composite of age + `time_of_purchase`; if not unique, use `main_id` auto-increment)


In [7]:
# Helper function to choose the first existing column from a list of candidates
def choose_first_existing(cands):
    for c in cands:
        if c in main_table.columns:
            return c
    return None

# Identify core columns
col_time   = next(c for c in main_table.columns if "time" in c)  # ex: time_of_purchase
col_gender = next(c for c in main_table.columns if "gender" in c)
col_amount = "purchase_amount"  # sesuai data yang sudah dibersihkan sebelumnya
# Hindari 'categorical_purchase_amount' tertangkap:
if "age" in main_table.columns:
    col_age = "age"
else:
    col_age = next(c for c in main_table.columns
                   if (c.endswith("age") or c.startswith("age")) and c != "categorical_purchase_amount")

col_payment = choose_first_existing(["payment_method","method","payment"])
col_product = choose_first_existing(["product_category","category","product"])
loc_candidates = [c for c in ["city","state","country","location","region"] if c in main_table.columns]
col_location = loc_candidates[0] if loc_candidates else None

fallback_pool = [c for c in main_table.columns
                 if c not in [col_payment,col_product,col_location,col_time,col_gender,col_amount,col_age,"categorical_purchase_amount"]]
fallback_cats = [c for c in fallback_pool if main_table[c].dtype=='object' or main_table[c].nunique()<=30]

# ===== DIMENSIONS =====
dim_gender = (main_table[[col_gender]].drop_duplicates().reset_index(drop=True)
              .rename(columns={col_gender:"gender"}))
dim_gender["gender_id"] = range(1, len(dim_gender)+1)

dim_payment = None
if col_payment is None and fallback_cats:
    col_payment = fallback_cats.pop(0)
if col_payment:
    dim_payment = (main_table[[col_payment]].drop_duplicates().reset_index(drop=True)
                   .rename(columns={col_payment:"payment_method"}))
    dim_payment["payment_method_id"] = range(1, len(dim_payment)+1)

dim_product = None
if col_product is None and fallback_cats:
    col_product = fallback_cats.pop(0)
if col_product:
    dim_product = (main_table[[col_product]].drop_duplicates().reset_index(drop=True)
                   .rename(columns={col_product:"product_category"}))
    dim_product["product_category_id"] = range(1, len(dim_product)+1)

dim_location = None
if col_location is None and fallback_cats:
    col_location = fallback_cats.pop(0)
if col_location:
    dim_location = (main_table[[col_location]].drop_duplicates().reset_index(drop=True)
                    .rename(columns={col_location:"location"}))
    dim_location["location_id"] = range(1, len(dim_location)+1)

dim_time = (pd.DataFrame({"time_of_purchase": main_table[col_time]})
            .drop_duplicates().dropna().reset_index(drop=True))
dim_time["date"]  = dim_time["time_of_purchase"].dt.date
dim_time["year"]  = dim_time["time_of_purchase"].dt.year
dim_time["month"] = dim_time["time_of_purchase"].dt.month
dim_time["day"]   = dim_time["time_of_purchase"].dt.day
dim_time["hour"]  = dim_time["time_of_purchase"].dt.hour
dim_time["time_id"] = range(1, len(dim_time)+1)

# ===== FACT =====
fact = main_table.copy()
fact = fact.merge(dim_gender, left_on=col_gender, right_on="gender", how="left")

if dim_payment is not None:
    fact = (fact.merge(dim_payment, left_on=col_payment, right_on="payment_method", how="left")
                .drop(columns=[col_payment]))

if dim_product is not None:
    # ✅ perbaikan utama: gunakan kolom sumber (col_product), bukan dim_product.columns[0]
    fact = (fact.merge(dim_product, left_on=col_product, right_on="product_category", how="left")
                .drop(columns=[col_product]))

if dim_location is not None:
    # ✅ perbaikan serupa untuk location
    fact = (fact.merge(dim_location, left_on=col_location, right_on="location", how="left")
                .drop(columns=[col_location]))

fact = fact.merge(dim_time[["time_of_purchase","time_id"]],
                  left_on=col_time, right_on="time_of_purchase", how="left")

# PK komposit vs surrogate
fact["_composite_key"] = fact[col_age].astype(str) + "|" + fact[col_time].astype(str)
is_unique = fact["_composite_key"].nunique() == len(fact)
if is_unique:
    fact.rename(columns={col_age:"age"}, inplace=True)
    fact["main_pk_age"]  = fact["age"]
    fact["main_pk_time"] = fact[col_time]
    fact.drop(columns=["_composite_key"], inplace=True)
else:
    fact["main_id"] = range(1, len(fact)+1)
    fact.rename(columns={col_age:"age"}, inplace=True)
    fact.drop(columns=["_composite_key"], inplace=True)

keep_cols = ["main_id"] if "main_id" in fact.columns else ["main_pk_age","main_pk_time"]
for c in ["age", col_amount, "categorical_purchase_amount", "gender_id", "time_id"]:
    if c not in keep_cols and c in fact.columns:
        keep_cols.append(c)
for c in ["payment_method_id","product_category_id","location_id"]:
    if c in fact.columns:
        keep_cols.append(c)

main_fact = fact[keep_cols].copy()
print("main_fact shape:", main_fact.shape)
display(main_fact.head())


main_fact shape: (859, 9)


,main_id,age,purchase_amount,categorical_purchase_amount,gender_id,time_id,payment_method_id,product_category_id,location_id
0,1,22,333.80,high,1,1.0,1,1,1
1,2,24,426.22,high,1,2.0,1,2,2
2,3,29,101.31,low,1,3.0,1,2,2
3,4,33,211.70,mid,1,4.0,1,3,3
4,5,45,487.95,high,2,5.0,1,1,2


#### **IV.B. Export all tables to CSV (for COPY process in PostgreSQL)**

In [8]:

def to_csv(df, name):
    path = TABLES_DIR / f"{name}.csv"
    df.to_csv(path, index=False)
    print("Wrote:", path)

to_csv(main_fact, "main_fact")
to_csv(dim_gender, "dim_gender")
to_csv(dim_time, "dim_time")
if 'dim_payment' in locals() and dim_payment is not None:
    to_csv(dim_payment, "dim_payment_method")
if 'dim_product' in locals() and dim_product is not None:
    to_csv(dim_product, "dim_product_category")
if 'dim_location' in locals() and dim_location is not None:
    to_csv(dim_location, "dim_location")


Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\main_fact.csv
Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\dim_gender.csv
Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\dim_time.csv
Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\dim_payment_method.csv
Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\dim_product_category.csv
Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\tables\dim_location.csv


---

# **V. RELATIONAL DATABASE & SQL**

This notebook **generates a `.sql` file** (DDL + COPY + test queries) for direct use.
Before running the `COPY` command, **move** the CSV files from the `tables/` folder to the **`/tmp/`** folder.


In [9]:

db_name = "gc2_consumer_behavior"

use_surrogate = "main_id" in main_fact.columns

lines = []
lines.append(textwrap.dedent(f'''
-- 1) CREATE DATABASE (jalankan dulu terpisah)
CREATE DATABASE {db_name};
\\c {db_name};
''').strip())

# Core dimensions
lines.append(textwrap.dedent('''
-- 2) DIM TABLES
CREATE TABLE dim_gender (
  gender_id INTEGER PRIMARY KEY,
  gender TEXT NOT NULL
);
''').strip())

lines.append(textwrap.dedent('''
CREATE TABLE dim_time (
  time_id INTEGER PRIMARY KEY,
  time_of_purchase TIMESTAMP NOT NULL,
  date DATE,
  year INT,
  month INT,
  day INT,
  hour INT
);
''').strip())

# optional dims exist?
has_payment = ('dim_payment' in locals()) and (dim_payment is not None)
has_product = ('dim_product' in locals()) and (dim_product is not None)
has_location = ('dim_location' in locals()) and (dim_location is not None)

if has_payment:
    lines.append(textwrap.dedent('''
CREATE TABLE dim_payment_method (
  payment_method_id INTEGER PRIMARY KEY,
  payment_method TEXT
);
''').strip())

if has_product:
    lines.append(textwrap.dedent('''
CREATE TABLE dim_product_category (
  product_category_id INTEGER PRIMARY KEY,
  product_category TEXT
);
''').strip())

if has_location:
    lines.append(textwrap.dedent('''
CREATE TABLE dim_location (
  location_id INTEGER PRIMARY KEY,
  location TEXT
);
''').strip())

# fact
if use_surrogate:
    lines.append(textwrap.dedent('''
CREATE TABLE main_fact (
  main_id SERIAL PRIMARY KEY,
  age INT NOT NULL,
  purchase_amount NUMERIC,
  categorical_purchase_amount TEXT,
  gender_id INT REFERENCES dim_gender(gender_id),
  time_id INT REFERENCES dim_time(time_id),
  payment_method_id INT REFERENCES dim_payment_method(payment_method_id),
  product_category_id INT REFERENCES dim_product_category(product_category_id),
  location_id INT REFERENCES dim_location(location_id)
);
''').strip())
else:
    lines.append(textwrap.dedent('''
CREATE TABLE main_fact (
  main_pk_age INT NOT NULL,
  main_pk_time TIMESTAMP NOT NULL,
  age INT NOT NULL,
  purchase_amount NUMERIC,
  categorical_purchase_amount TEXT,
  gender_id INT REFERENCES dim_gender(gender_id),
  time_id INT REFERENCES dim_time(time_id),
  payment_method_id INT REFERENCES dim_payment_method(payment_method_id),
  product_category_id INT REFERENCES dim_product_category(product_category_id),
  location_id INT REFERENCES dim_location(location_id),
  PRIMARY KEY (main_pk_age, main_pk_time)
);
''').strip())

# COPY
copy_lines = [
    "COPY dim_gender(gender, gender_id) FROM '/tmp/dim_gender.csv' CSV HEADER;",
    "COPY dim_time(time_of_purchase, date, year, month, day, hour, time_id) FROM '/tmp/dim_time.csv' CSV HEADER;"
]
if has_payment:
    copy_lines.append("COPY dim_payment_method(payment_method, payment_method_id) FROM '/tmp/dim_payment_method.csv' CSV HEADER;")
if has_product:
    copy_lines.append("COPY dim_product_category(product_category, product_category_id) FROM '/tmp/dim_product_category.csv' CSV HEADER;")
if has_location:
    copy_lines.append("COPY dim_location(location, location_id) FROM '/tmp/dim_location.csv' CSV HEADER;")

if use_surrogate:
    copy_lines.append("COPY main_fact(main_id, age, purchase_amount, categorical_purchase_amount, gender_id, time_id, payment_method_id, product_category_id, location_id) FROM '/tmp/main_fact.csv' CSV HEADER;")
else:
    copy_lines.append("COPY main_fact(main_pk_age, main_pk_time, age, purchase_amount, categorical_purchase_amount, gender_id, time_id, payment_method_id, product_category_id, location_id) FROM '/tmp/main_fact.csv' CSV HEADER;")

lines.append("-- 3) COPY CSVs into tables (pastikan CSV sudah ada di /tmp)\\n" + "\\n".join(copy_lines))

# TEST QUERIES
lines.append(textwrap.dedent('''
-- 4) Database Testing

-- a) Total Purchase Amount per Gender for Age <= 30
SELECT g.gender, SUM(f.purchase_amount) AS total_purchase_amount
FROM main_fact f
JOIN dim_gender g ON f.gender_id = g.gender_id
JOIN dim_time t ON f.time_id = t.time_id
WHERE f.age <= 30
GROUP BY g.gender
ORDER BY g.gender;

-- b) Summary statistics (avg, min, max) by one categorical column (e.g., product_category)
SELECT p.product_category,
       AVG(f.purchase_amount) AS avg_amount,
       MIN(f.purchase_amount) AS min_amount,
       MAX(f.purchase_amount) AS max_amount
FROM main_fact f
JOIN dim_product_category p ON f.product_category_id = p.product_category_id
GROUP BY p.product_category
ORDER BY avg_amount DESC;
''').strip())

sql_text = "\\n\\n".join(lines)

sql_path = PROJECT_DIR / "GC2_Consumer_Behavior_SQL.sql"
sql_path.write_text(sql_text, encoding="utf-8")
print("Wrote SQL:", sql_path)


Wrote SQL: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\GC2_Consumer_Behavior_SQL.sql


---

# **VI. DATABASE TESTING: RUN QUERIES AND EXPORT RESULTS (PANDAS VERSION)**

>The assignment requires two data files containing the query results. Below, we replicate the SQL >logic using Pandas so the results can be exported directly from the notebook.
> (On the system, also run the same SQL queries in PostgreSQL)


In [10]:

res_a = (main_fact.merge(dim_gender, on="gender_id")
                   .query("age <= 30")
                   .groupby("gender", as_index=False)["purchase_amount"]
                   .sum()
                   .rename(columns={"purchase_amount":"total_purchase_amount"}))

res_a_path = QUERY_OUT_DIR / "query_a_total_purchase_by_gender_age_le_30.csv"
res_a.to_csv(res_a_path, index=False)
print("Wrote:", res_a_path)
display(res_a)

if 'product_category_id' in main_fact.columns:
    joined = main_fact.merge(dim_product, on="product_category_id")
    group_col, label = "product_category", "by_product_category"
elif 'payment_method_id' in main_fact.columns:
    joined = main_fact.merge(dim_payment, on="payment_method_id")
    group_col, label = "payment_method", "by_payment_method"
else:
    joined = main_fact.merge(dim_location, on="location_id")
    group_col, label = "location", "by_location"

res_b = (joined.groupby(group_col)["purchase_amount"]
               .agg(avg_amount="mean", min_amount="min", max_amount="max")
               .reset_index()
               .sort_values("avg_amount", ascending=False))

res_b_path = QUERY_OUT_DIR / f"query_b_stats_{label}.csv"
res_b.to_csv(res_b_path, index=False)
print("Wrote:", res_b_path)
display(res_b.head(10))


Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\query_outputs\query_a_total_purchase_by_gender_age_le_30.csv


,gender,total_purchase_amount
0,Female,51052.07
1,Male,45716.22


Wrote: c:\Users\User\Desktop\consumer behaviour\gc2_consumer_behavior_project\query_outputs\query_b_stats_by_product_category.csv


,product_category,avg_amount,min_amount,max_amount
2,Single,280.966919,50.79,498.23
1,Married,277.024032,51.22,498.33
0,Divorced,276.544694,51.86,494.34
3,Widowed,269.040190,50.78,497.80


> **Final Notes:** Make sure to copy all CSV files from the `tables/` folder to `/tmp/` before running the `COPY` command in PostgreSQL.

---

# **VII. CONCLUSION**

**Analytical summary.** The dataset (964 rows) shows adult-heavy demographics (age mean = 34) with moderate missingness in marketing signals (e.g., *engagement_with_ads*, *social_media_influence*). Key fields require normalization: `purchase_amount` must be cast from currency strings to numeric; `time_of_purchase` must be parsed to datetime; and `gender` needs standardization to Male/Female. Purchase behavior is granular (many unique price points) and likely right‑skewed; frequency centers around 7 purchases; average brand loyalty and product ratings hover near 3/5, satisfaction is mid‑range (5.4/10). Channel/device usage suggests multi‑touch journeys (Mixed channel common) with a Desktop tilt. Category breadth is wide (24 product categories with a long tail; Electronics leads), which is ideal for dimension modeling and rollups.

**Business impact & recommendations.**
1) **Segmentation & targeting.** Use the 3NF model to segment by *product_category*, *payment_method*, *channel*, *device*, *age bands*, and *discount_sensitivity*. Prioritize *Very Sensitive* discount cohorts with targeted promo cadence.  
2) **Offer & pricing strategy.** Given right‑skewed spend, deploy tiered promotions and *AOV* boosters (bundles/cross‑sell) for mid‑/high‑spend buckets; cap discounts for highly sensitive segments to protect margin.  
3) **Channel orchestration.** “Mixed” dominance implies multi‑channel paths; coordinate creative and frequency across web/email/paid to reduce fatigue and attribute uplift correctly.  
4) **Device experience.** With Desktop leading, optimize web UX (checkout friction, payment options); monitor conversion gaps on Mobile and test responsive improvements.  
5) **Loyalty & retention.** Mid‑level loyalty/satisfaction signals headroom—expand post‑purchase journeys (how‑to content, proactive service), and test loyalty tiers to lift repeat purchase and reduce return propensity.  
6) **Category strategy.** Long‑tail categories warrant category‑level creatives and inventory focus; keep *Electronics* as a performance anchor while testing lift for adjacent categories.  
7) **Returns management.** Track `return_rate` by category/channel/payment to spot policy or quality issues; implement pre‑purchase expectation setting to lower returns.

**Measurement plan (SQL over the 3NF model).**
- Core KPIs: *AOV*, total and net revenue, repeat rate, 30‑/60‑day repurchase, return %, NPS/satisfaction index, and discount lift.  
- Standardize definitions in views (e.g., `vw_daily_sales`, `vw_category_kpis`) to ensure one‑truth reporting across teams.  
- Maintain conformed dimensions (`dim_time`, `dim_gender`, `dim_payment_method`, `dim_product_category`, `dim_location`) for consistent slicing.

**Next steps & data quality.**
- Validate anomalies (e.g., *marital_status* “Widowed” dominance; *occupation* values like “High”) with source owners; fix mapping.  
- Enrich with marketing context: campaign IDs, acquisition channels, and cohort tags for attribution and LTV.  
- Automate the load (COPY or ELT) and stand up BI dashboards for Marketing Ops; schedule data quality checks (nulls, ranges, referential integrity).

*Outcome:* a repeatable, analytics‑ready warehouse that enables marketing to allocate budget, tailor offers, and track KPIs reliably across channels and segments.
